<a href="https://colab.research.google.com/github/MuhamaadUsman/Model-Notebooks/blob/main/BERT%20For%20Custom%20NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers seqeval[gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=9188764f8c2983f181ad317ab253642a5964a1ddc8cba677606d747051031c7a
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
data = pd.read_excel('/content/output_label_token.xlsx')

data.head()

,Para #,Token,Labels
0,para :1,item,O
1,NaN,10,O
2,NaN,.,O
3,NaN,,O
4,NaN,directors,O


In [ ]:
data.count()

Para #       14
Token     24753
Labels    24753
dtype: int64

In [ ]:
data['Labels'] = data['Labels'].str.replace('B-Position', 'B-position')

In [ ]:
print("Number of tags: {}".format(len(data.Labels.unique())))
frequencies = data.Labels.value_counts()
frequencies

Number of tags: 6


O             24441
B-position       91
I-position       87
I-name           58
B-name           38
B-age            38
Name: Labels, dtype: int64

In [ ]:
data.to_excel('/content/output_label_token.xlsx', index=False)


In [ ]:
label2id = {k: v for v, k in enumerate(data.Labels.unique())}
id2label = {v: k for v, k in enumerate(data.Labels.unique())}
label2id

{'O': 0,
 'B-name': 1,
 'I-name': 2,
 'B-position': 3,
 'I-position': 4,
 'B-age': 5}

In [ ]:
data = data.fillna(method='ffill')
data.head()

,Para #,Token,Labels
0,para :1,item,O
1,para :1,10,O
2,para :1,.,O
3,para :1,,O
4,para :1,directors,O


In [ ]:
data['sentence'] = data[['Para #','Token','Labels']].groupby(['Para #'])['Token'].transform(lambda x: ' '.join(x))
data['word_labels'] = data[['Para #','Token','Labels']].groupby(['Para #'])['Labels'].transform(lambda x: ','.join(x))
data.head()

,Para #,Token,Labels,sentence,word_labels
0,para :1,item,O,"item 10 . directors , executive offi...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,para :1,10,O,"item 10 . directors , executive offi...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2,para :1,.,O,"item 10 . directors , executive offi...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3,para :1,,O,"item 10 . directors , executive offi...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
4,para :1,directors,O,"item 10 . directors , executive offi...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."


In [ ]:
data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data.head()

,sentence,word_labels
0,"item 10 . directors , executive offi...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,"item 10 . directors , executive officers and c...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2,"item 10 . directors , executive officers and c...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3,"item 10 . directors , executive of...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
4,"item 10 . directors , executive officers and c...","O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."


In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2
EPOCHS = 250
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# # Assuming data['sentence'] contains pre-tokenized sentences


In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]
        word_labels = self.data.word_labels[index]
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)

        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)
        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [ ]:
train_size = 0.9
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (14, 2)
TRAIN Dataset: (13, 2)
TEST Dataset: (1, 2)


In [ ]:
training_set[0]


{'ids': tensor([  101,  8875,  2184,  1012,  5501,  1010,  3237,  3738,  1998,  5971,
         10615,  1012,  1996,  2206,  2795,  4520,  5743,  1996,  2171,  1010,
          2287,  1998,  2597,  1997,  2169,  1997,  2256,  3237,  3738,  1998,
          5501,  2004,  1997,  2285,  2861,  1010,  2418,  1012,  5501,  1998,
          3237,  3738,  2171,  2287,  2597,  4775, 21701, 28522,  4779,  5764,
          2708,  3237,  2961,  1010,  2343,  1010,  2708,  3361,  2961,  1998,
          2472,  1996,  2206,  5577,  1996,  2449,  3325,  1997,  2169,  1997,
          2256,  5501,  1998,  3237,  3738,  1010,  2164,  2060,  5501, 19801,
          2218,  1999,  2270,  7316,  3316,  1010,  2065,  2151,  1024,  4775,
         21701, 28522,  4779,  1010,  2708,  3237,  2961,  1010,  2708,  3361,
          2961,  1998,  2472,  1011,  4775, 21701, 28522,  4779,  2038,  2871,
          2086,  1997,  2658,  3325,  1999,  2449,  2458, 13252,  1010,  2892,
          3675, 28585,  1998, 19530,  1010,  

In [ ]:
training_set[0]["ids"]

tensor([  101,  8875,  2184,  1012,  5501,  1010,  3237,  3738,  1998,  5971,
        10615,  1012,  1996,  2206,  2795,  4520,  5743,  1996,  2171,  1010,
         2287,  1998,  2597,  1997,  2169,  1997,  2256,  3237,  3738,  1998,
         5501,  2004,  1997,  2285,  2861,  1010,  2418,  1012,  5501,  1998,
         3237,  3738,  2171,  2287,  2597,  4775, 21701, 28522,  4779,  5764,
         2708,  3237,  2961,  1010,  2343,  1010,  2708,  3361,  2961,  1998,
         2472,  1996,  2206,  5577,  1996,  2449,  3325,  1997,  2169,  1997,
         2256,  5501,  1998,  3237,  3738,  1010,  2164,  2060,  5501, 19801,
         2218,  1999,  2270,  7316,  3316,  1010,  2065,  2151,  1024,  4775,
        21701, 28522,  4779,  1010,  2708,  3237,  2961,  1010,  2708,  3361,
         2961,  1998,  2472,  1011,  4775, 21701, 28522,  4779,  2038,  2871,
         2086,  1997,  2658,  3325,  1999,  2449,  2458, 13252,  1010,  2892,
         3675, 28585,  1998, 19530,  1010,  1998,  2248, 13797])

In [ ]:
# print the first 30 tokens and corresponding labels
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:30]), training_set[0]["targets"][:30]):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

[CLS]       O
item        O
10          O
.           O
directors   O
,           O
executive   O
officers    O
and         O
corporate   O
governance  O
.           O
the         O
following   O
table       O
sets        O
forth       O
the         O
name        O
,           O
age         O
and         O
position    O
of          O
each        O
of          O
our         O
executive   O
officers    O
and         O


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased',
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)
model.to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(1.7477, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 6])

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 1.8572938442230225
Training loss epoch: 1.794210365840367
Training accuracy epoch: 0.19196428571428573
Training epoch: 2
Training loss per 100 training steps: 1.8485840559005737
Training loss epoch: 1.811699628829956
Training accuracy epoch: 0.17243303571428573
Training epoch: 3
Training loss per 100 training steps: 1.800255537033081
Training loss epoch: 1.7856673683438982
Training accuracy epoch: 0.18303571428571427
Training epoch: 4
Training loss per 100 training steps: 1.7502318620681763
Training loss epoch: 1.8006633009229387
Training accuracy epoch: 0.17466517857142858
Training epoch: 5
Training loss per 100 training steps: 1.82045316696167
Training loss epoch: 1.818732840674264
Training accuracy epoch: 0.15792410714285715
Training epoch: 6
Training loss per 100 training steps: 1.7541179656982422
Training loss epoch: 1.7974542890276228
Training accuracy epoch: 0.17410714285714285
Training epoch: 7
Training loss per 100 traini

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [ ]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 1.9051517248153687
Validation Loss: 1.9051517248153687
Validation Accuracy: 0.078125


In [ ]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

              precision    recall  f1-score   support

         age       0.15      0.57      0.24         7
        name       0.00      0.00      0.00         7
    position       0.07      0.22      0.10         9

   micro avg       0.06      0.26      0.10        23
   macro avg       0.07      0.26      0.11        23
weighted avg       0.07      0.26      0.11        23



In [ ]:
sentence = """
name: dayong sun    age: 43    position: president, chief executive officer, treasurer, secretary and director    each director serves
until the next annual meeting of shareholders and until his/her successor shall have been elected and qualified.
"""

inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")

# move to gpu
ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
# forward pass
outputs = model(ids, mask)
logits = outputs[0]

active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [id2label[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

word_level_predictions = []
for pair in wp_preds:
  if (pair[0].startswith(" ##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
    # skip prediction
    continue
  else:
    word_level_predictions.append(pair[1])

# we join tokens, if they are not special ones
str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
print(str_rep)
print(word_level_predictions)

name : dayong sun age : 43 position : president , chief executive officer , treasurer , secretary and director each director serves until the next annual meeting of shareholders and until his / her successor shall have been elected and qualified .
['B-age', 'I-position', 'B-name', 'B-name', 'B-age', 'B-age', 'I-position', 'I-position', 'B-age', 'I-position', 'I-position', 'O', 'B-age', 'B-age', 'I-name', 'O', 'I-position', 'O', 'B-age', 'I-name', 'B-age', 'B-age', 'I-position', 'B-position', 'B-position', 'B-age', 'B-age', 'B-age', 'I-name', 'B-name', 'B-age', 'O', 'B-position', 'B-age', 'B-name', 'B-name', 'B-age', 'B-age', 'B-age', 'B-age', 'I-name', 'I-name', 'B-age', 'O']
